In [2]:
! pip install pillow torch numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 7.5 MB/s eta 0:00:00 MB/s eta 0:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 MB 32.7 MB/s eta 0:00:00m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 39.0 MB/s eta 0:00:0031m42.1 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 33.6 MB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 39.0 MB/s eta 0:00:0031m46.5 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 36.9 MB/s eta 0:00:00


In [3]:
! pip install matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.5/159.5 kB 2.7 MB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 7.4 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 15.4 MB/s eta 0:00:000:00:01m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 kB 8.3 MB/s eta 0:00:00


In [4]:
import os
import zipfile
from PIL import Image
import torch
import time
import numpy as np

In [5]:
# Unzip file

# Path to the zip file
zip_file = "GTA5.zip"

# Directory where you want to extract the contents
extract_dir = 'GTA5'

# Create the extract directory if it doesn't exist
os.makedirs(extract_dir, exist_ok=True)

# Extract the contents of the zip file
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("Extraction complete.")

Extraction complete.


In [6]:
label_dir = 'GTA5/GTA5/labels'
label_files = []

for file_name in os.listdir(label_dir):
    file_path = os.path.join(label_dir, file_name)
    if os.path.isfile(file_path):
        label_files.append(file_name)

In [7]:
len(label_files)

2500

In [8]:
# Define the color to label mapping dictionary
color_to_label = {
    (128, 64, 128): 0,  # road
    (244, 35, 232): 1,  # sidewalk
    (70, 70, 70): 2,  # building
    (102, 102, 156): 3,  # wall
    (190, 153, 153): 4,  # fence
    (153, 153, 153): 5,  # pole
    (250, 170, 30): 6,  # light
    (220, 220, 0): 7,  # sign
    (107, 142, 35): 8,  # vegetation
    (152, 251, 152): 9,  # terrain
    (70, 130, 180): 10,  # sky
    (220, 20, 60): 11,  # person
    (255, 0, 0): 12,  # rider
    (0, 0, 142): 13,  # car
    (0, 0, 70): 14,  # truck
    (0, 60, 100): 15,  # bus
    (0, 80, 100): 16,  # train
    (0, 0, 230): 17,  # motorcycle
    (119, 11, 32): 18,  # bicycle
    (0, 0, 0): 255  # noclass
}


# Convert the color to label mapping to tensor form
color_list = list(color_to_label.keys())
label_list = list(color_to_label.values())

# Convert to torch tensors
color_tensor = torch.tensor(color_list, dtype=torch.long)
label_tensor = torch.tensor(label_list, dtype=torch.long)

In [9]:
# Function to map RGB triplet to greyscale label
def rgb_to_label(label_name, color_tensor, label_tensor):
    
    # pick the path
    label_path = os.path.join(label_dir, label_name)
    
    # extract the image
    label = Image.open(label_path).convert('RGB')
    
    # transfrom to tensor
    tensor = torch.Tensor(np.array(label)).long()
    
    # Reshape the input tensor to (H*W, 3) for easier comparison
    reshaped_tensor = tensor.view(-1, 3)
    
    # Initialize the output label tensor
    label_map = torch.zeros(reshaped_tensor.shape[0], dtype=torch.long)
    
    # Compare each pixel with the color tensor and assign labels
    for i, color in enumerate(color_tensor):
        mask = torch.all(reshaped_tensor == color, dim=1)
        label_map[mask] = label_tensor[i]
    
    # Reshape the label map back to the original image shape
    label_map = label_map.view(tensor.shape[0], tensor.shape[1])

    # Convert tensor (label_map) to a grayscale image
    label_map_np = label_map.cpu().numpy().astype(np.uint8)
    gray_image = Image.fromarray(label_map_np, mode='L')

    # Make a directory if it does not exist
    save_dir = 'GTA5/GTA5/masks'
    os.makedirs(save_dir, exist_ok=True)

    # Save the image in the directory
    save_path = os.path.join(save_dir, label_name)
    gray_image.save(save_path)
    
    print(f"Image saved to {save_path}")

In [10]:
for label_name in label_files:
    rgb_to_label(label_name, color_tensor, label_tensor)

Image saved to GTA5/GTA5/masks/02296.png
Image saved to GTA5/GTA5/masks/01947.png
Image saved to GTA5/GTA5/masks/00481.png
Image saved to GTA5/GTA5/masks/01953.png
Image saved to GTA5/GTA5/masks/00495.png
Image saved to GTA5/GTA5/masks/02282.png
Image saved to GTA5/GTA5/masks/00324.png
Image saved to GTA5/GTA5/masks/01984.png
Image saved to GTA5/GTA5/masks/00442.png
Image saved to GTA5/GTA5/masks/02255.png
Image saved to GTA5/GTA5/masks/02241.png
Image saved to GTA5/GTA5/masks/01990.png
Image saved to GTA5/GTA5/masks/00456.png
Image saved to GTA5/GTA5/masks/01748.png
Image saved to GTA5/GTA5/masks/00330.png
Image saved to GTA5/GTA5/masks/01006.png
Image saved to GTA5/GTA5/masks/00318.png
Image saved to GTA5/GTA5/masks/02269.png
Image saved to GTA5/GTA5/masks/01760.png
Image saved to GTA5/GTA5/masks/01774.png
Image saved to GTA5/GTA5/masks/01012.png
Image saved to GTA5/GTA5/masks/00683.png
Image saved to GTA5/GTA5/masks/02094.png
Image saved to GTA5/GTA5/masks/02080.png
Image saved to G

In [12]:
# Zip the directory GTA5/GTA5
import shutil
shutil.make_archive('GTA5/GTA5', 'zip', 'GTA5/GTA5')

'/Users/tommasomazzarini/Code/Project-MLDL/GTA5/GTA5.zip'